In [49]:
import os
import glob

data=[]

for fi in glob.glob("/content/*.jsonl"):
    with open(fi, "r+") as f:
      data.extend([json.loads(l) for l in f])

df = pd.DataFrame(data)
sorted_df = df.sort_values(by=['id'], ascending=True)
#sorted_df.reset_index()
sorted_df=sorted_df.drop_duplicates()
len(sorted_df)
#sorted_df.loc["98720"]


12140

In [50]:
sorted_df.to_csv("merged.csv")

In [1]:
# Install the transformers library
!pip install datasets transformers==4.28.0
!pip install --upgrade accelerate

In [51]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [52]:
# Load tokenizer and model, create trainer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [59]:
df_pred = pd.read_csv("merged.csv")
pred_texts = df_pred["text"].dropna().astype('str').tolist()
len(pred_texts)

12140

In [60]:
#tokenizing
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [61]:
predictions = trainer.predict(pred_dataset)

In [63]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [ ]:
# Create DataFrame with texts, predictions, labels, and scores
id_list=df_pred["id"].tolist()
img_list=df_pred["img"].tolist()
label_list=df_pred["label"].tolist()

df = pd.DataFrame(list(zip(id_list,img_list,label_list,pred_texts,labels,scores)), columns=['id','img','hateful_label','text','Sentiment_Label','Sentiment_Score'])
df.head()

In [69]:
#the label for the meme (0=not-hateful, 1=hateful)
mappings = {
    0: 'not-hateful',
    1: 'hateful'
}

df['hateful_label'] = df['hateful_label'].map(mappings)

In [82]:
df.head()
df.to_csv("sentiment_output.csv")

In [74]:
df.head()

,id,img,hateful_label,text,Sentiment_Label,Sentiment_Score
0,1235,img/01235.png,hateful,when you're feeling horny asf but your habibi ...,POSITIVE,0.914592
1,1236,img/01236.png,not-hateful,meanwhile at the isis strip club,POSITIVE,0.993525
2,1243,img/01243.png,not-hateful,"when your human says ""who' s a good girl?"" and...",POSITIVE,0.995225
3,1245,img/01245.png,not-hateful,and that was the last nativity play my son was...,NEGATIVE,0.984378
4,1247,img/01247.png,hateful,you can't be racist if there is no other race,NEGATIVE,0.987061


In [85]:
correct=len(df[(df['hateful_label']=="hateful") & (df['Sentiment_Label']=='NEGATIVE')])
total_hateful=len(df[(df['hateful_label']=="hateful")])
total_not_hateful=len(df[(df['hateful_label']=="not-hateful")])
print(total_hateful)#total no of data points for which sentiment is negative and hateful annoatation is true
print(total_not_hateful)#total data points

4506
7634


In [86]:
per=correct*100/total_hateful
per

66.73324456280515

In [ ]:
#Only 66.7% cases of sentiment detection being useful.